In [1]:
import pandas as pd
from sqlalchemy import create_engine
from pyhive import presto
from tqdm import tqdm,trange,tqdm_notebook
from joblib  import Parallel, delayed

In [2]:
def create_temp_list(id_info):
    return test[test.id == id_info]

In [3]:
length = 125871085

In [4]:
test1 = pd.read_pickle('req_mapp.pkl')

In [5]:
engine_presto = presto.connect('133.186.168.10')
while len(test1) < length:
    del test1
    test1 = pd.read_pickle('req_mapp.pkl')
    query = """
    select *
    from MEUMS_COMP_REQ_MAPP
    limit 500000
    """
    test = pd.read_sql_query(query, engine_presto)
    print('presto call success!',len(test))
    
    temp_list = []
    id_info_list = list(set(test.id).difference(set(test1.id)))
    print('id_info_list length : ', len(id_info_list))
    temp_list += Parallel(n_jobs = -1)(delayed(create_temp_list)(id_info) for id_info in tqdm_notebook(id_info_list))
        
    if len(temp_list) == 0:
        print('overlap occured!')
        continue
    comp = pd.concat(temp_list).reset_index().drop(['index'], axis = 1)
    del temp_list
    
    test1 = pd.concat([test1, comp]).reset_index().drop(['index'], axis = 1)
    test1.to_pickle('req_mapp.pkl')
    del comp
    print('pickle create success!', 'pickle dataframe length : ', len(test1))

presto call success! 500000
id_info_list length :  301578


KeyboardInterrupt: 

In [13]:
del temp_list

In [9]:
count = 0 
for name in tqdm_notebook(range(0,100000000000)):
    count+=1
count

KeyboardInterrupt: 